In [ ]:
import os
from typing import Annotated, Literal

from pydantic import BaseModel

from autogen.agentchat import ConversableAgent, UserProxyAgent
from autogen.tools.dependency_injection import BaseContext, ChatContext, Depends, inject_params

In [2]:
class Account(BaseContext, BaseModel):
    username: str
    password: str
    currency: Literal["USD", "EUR"] = "USD"


alice_account = Account(username="alice", password="password123")
bob_account = Account(username="bob", password="password456")

account_ballace_dict = {
    (alice_account.username, alice_account.password): 300,
    (bob_account.username, bob_account.password): 200,
}

In [3]:
def _verify_account(account: Account):
    if (account.username, account.password) not in account_ballace_dict:
        raise ValueError("Invalid username or password")


def _get_balance(account: Account):
    _verify_account(account)
    return f"Your balance is {account_ballace_dict[(account.username, account.password)]}{account.currency}"

In [4]:
config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]
agent = ConversableAgent(
    name="agent",
    llm_config={"config_list": config_list},
)

In [5]:
def get_balance_1(
    account: Annotated[Account, Depends(bob_account)],
    chat_context: Annotated[ChatContext, "description of chat context"],
) -> str:
    print(f"Chatcontext: {chat_context.chat_messages}")
    return _get_balance(account)

In [6]:
func = inject_params(get_balance_1)

In [7]:
from functools import wraps
from typing import Any

chat_contexts = {"chat_context": ChatContext(agent=agent)}


def update_chat_context(func, agent: "ConversableAgent"):
    @wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        assert len(kwargs) == 0
        for value in kwargs.values():
            if isinstance(value, ChatContext):
                value.chat_messages = agent.chat_messages()

        return func(*args, **kwargs, **chat_contexts)

    return wrapper

In [ ]:
import inspect

func = update_chat_context(func, agent)

inspect.signature(func)

In [ ]:
func()

## Real chat

In [10]:
config_list = [{"model": "gpt-4o-mini", "api_key": os.environ["OPENAI_API_KEY"]}]
agent = ConversableAgent(
    name="agent",
    llm_config={"config_list": config_list},
)
user_proxy = UserProxyAgent(
    name="user_proxy_1",
    human_input_mode="NEVER",
    llm_config=False,
)

In [11]:
from logging import Logger


@user_proxy.register_for_execution()
@agent.register_for_llm(description="Get balance")
def get_balance_1(
    account: Annotated[Account, Depends(bob_account)],
    # chat_context: Annotated[ChatContext, "Injected chat context"],
    chat_context: ChatContext,
    logger: Logger,
) -> str:
    # _setup_logger_for_notebook(logger)
    print(f"context: {chat_context.chat_messages}")
    print(f"Last message: {chat_context.last_message}")
    logger.info("Hello, World!")
    return _get_balance(account)

In [ ]:
user_proxy.initiate_chat(agent, message="Get users balance, If balance is less then 500, get it again", max_turns=5)